In [73]:
import re
import pymongo
import findspark 
findspark.init()
import pyspark
from pyspark.sql.functions import col, udf
import pymongo_spark
pymongo_spark.activate()
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, CountVectorizer, IDFModel, StopWordsRemover
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, GBTClassifier, RandomForestClassificationModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, ParamGridBuilder, TrainValidationSplit

In [7]:
spark = pyspark.sql.SparkSession.builder \
            .master("local[4]") \
            .appName('Testing Spark Mongo') \
            .getOrCreate()

In [8]:
people = spark.createDataFrame([("Bilbo Baggins",  50), ("Gandalf", 1000), ("Thorin", 195), ("Balin", 178), ("Kili", 77),
   ("Dwalin", 169), ("Oin", 167), ("Gloin", 158), ("Fili", 82), ("Bombur", None)], ["name", "age"])

In [22]:
people.show()

+-------------+----+
|         name| age|
+-------------+----+
|Bilbo Baggins|  50|
|      Gandalf|1000|
|       Thorin| 195|
|        Balin| 178|
|         Kili|  77|
|       Dwalin| 169|
|          Oin| 167|
|        Gloin| 158|
|         Fili|  82|
|       Bombur|null|
+-------------+----+



In [37]:
path = '../kafka_files/twitterstream_0.jsonl'
schema = schema = StructType([
        StructField('Text' , StringType(), False),
    ])
df = spark.read.json(path, columnNameOfCorruptRecord='Text')
df.printSchema()
df.show(1, truncate=False)

root
 |-- Text: string (nullable = true)

+----------------------------------------------------------------------------------------------------------------------------+
|Text                                                                                                                        |
+----------------------------------------------------------------------------------------------------------------------------+
|"\"Weekly Specials: Worn 2017 Air Jordan IV Retro Motorsport Sz 11 w/ Box In Good Wearable\\u2026 https://t.co/cgdDiIKaDR\""|
+----------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row



In [50]:
def filter_ads(text):
    return 'https' not in text
ads_filter = udf(filter_ads, BooleanType())

In [51]:
ads_free = df.filter(ads_filter(df.Text))
ads_free.show()

+--------------------+
|                Text|
+--------------------+
|"\"@AmateurHookup...|
|"\"i can't even e...|
|"\"@McHavy @StayC...|
+--------------------+



In [57]:
def preprocess(text):
    words = re.sub("[^a-zA-Z]", " ", text).lower().split()
    return words
pp_udf = udf(preprocess, ArrayType(StringType()))
words = ads_free.withColumn('Words', pp_udf(ads_free.Text))
words.show()

+--------------------+--------------------+
|                Text|               Words|
+--------------------+--------------------+
|"\"@AmateurHookup...|[amateurhookup, f...|
|"\"i can't even e...|[i, can, t, even,...|
|"\"@McHavy @StayC...|[mchavy, staycare...|
+--------------------+--------------------+



In [60]:
#remove stop words
remover = StopWordsRemover(inputCol="Words", outputCol="filtered")
removed = remover.transform(words)
removed.show()


+--------------------+--------------------+--------------------+
|                Text|               Words|            filtered|
+--------------------+--------------------+--------------------+
|"\"@AmateurHookup...|[amateurhookup, f...|[amateurhookup, f...|
|"\"i can't even e...|[i, can, t, even,...|[even, explain, s...|
|"\"@McHavy @StayC...|[mchavy, staycare...|[mchavy, staycare...|
+--------------------+--------------------+--------------------+



In [61]:
params_path = '../tmp/{}'

In [66]:
#Load trained hashing frequency and transform
hf_path = params_path.format('hf')
hashingTF = HashingTF.load(hf_path)
featureized = hashingTF.transform(removed)
featureized.show()

+--------------------+--------------------+--------------------+--------------------+
|                Text|               Words|            filtered|         rawFeatures|
+--------------------+--------------------+--------------------+--------------------+
|"\"@AmateurHookup...|[amateurhookup, f...|[amateurhookup, f...|(200,[16,33,61],[...|
|"\"i can't even e...|[i, can, t, even,...|[even, explain, s...|(200,[2,6,25,60,7...|
|"\"@McHavy @StayC...|[mchavy, staycare...|[mchavy, staycare...|(200,[60,73,74,77...|
+--------------------+--------------------+--------------------+--------------------+



In [67]:
#Load trained hashing frequency and transform
idf_path = params_path.format('idfmodel')
idfmodel = IDFModel.load(idf_path)
result = idfmodel.transform(featureized)
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                Text|               Words|            filtered|         rawFeatures|            features|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|"\"@AmateurHookup...|[amateurhookup, f...|[amateurhookup, f...|(200,[16,33,61],[...|(200,[16,33,61],[...|
|"\"i can't even e...|[i, can, t, even,...|[even, explain, s...|(200,[2,6,25,60,7...|(200,[2,6,25,60,7...|
|"\"@McHavy @StayC...|[mchavy, staycare...|[mchavy, staycare...|(200,[60,73,74,77...|(200,[60,73,74,77...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [77]:
#load rf model and predict
rf_path = params_path.format('rf')
rf = RandomForestClassificationModel.load(rf_path)
prediction = rf.transform(result)
prediction.select('rawPrediction', 'prediction').show()

+--------------------+----------+
|       rawPrediction|prediction|
+--------------------+----------+
|[50.9842332206184...|       0.0|
|[55.7356588277157...|       0.0|
|[50.8181693836755...|       0.0|
+--------------------+----------+



In [87]:
path_to_save = '../tmp/twitterstream_test_prediction.json'
prediction.write.json(path_to_save)


In [88]:
test = spark.read.json(path_to_save)
test.printSchema()
test.show()

root
 |-- Text: string (nullable = true)
 |-- Words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- prediction: double (nullable = true)
 |-- probability: struct (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- rawFeatures: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |   